# Tarea 3: Relación entre efectividad y eficicencia en las búsquedas aproximadas del vecino más cercano.
## CC5213 - Recuperación de Información Multimedia

### Alumno: Benjamín del Pino B.
### Fecha: 31 de mayo de 2021

## Objetivo
El objetivo de esta tarea es estudiar la relación entre efectividad y eficiencia en las busquedas aproximadas del vecino más cervano (NN).

En particular se van a analizar dos conjuntos de datos que llamaremos **MEL** (descriptores de audio) y **VGG** (descriptores de video). Cada uno se compone de un conjunto de vectores **Q** y **R**. Para cada vector **q** de **Q** se desea encontrar su vecino más cercano **r** en **R** tal que la **d(q,r) $\leq$ d(q,x)** para todo vector **x** de **R** (en donde **d** corresponde a la distancia euclidiana).

## Técnicas a utilizar
1. **Randomized KD-Tree**
2. **Hierarchical K-Means Tree**
3. **Linear Scan sobre PCA**

Como línea base de efectividad y eficiencia se usará el algoritmo de fuerza bruta **Linear Scan**.

Para evaluar los resultados de una busqueda aproximada se deberán calcular dos indicadores:
- **Efectividad**
- **Eficiencia**.


### *El código de esta tarea fue extraido principalmente de los anexos que subió el profesor Juan Manuel Barrios en la semana 6 del curso CC5213.*

In [1]:
import os
import numpy

def load_file(filename, dimensions):
    assert os.path.isfile(filename), "no existe archivo " + filename
    mat = numpy.fromfile(filename, dtype=numpy.float32)
    return numpy.reshape(mat, (-1, dimensions))

"""
    Esta clase representa a las databases que serán analizadas
"""
class Dataset:   
    def __init__(self, nombre, archivo_q, archivo_r, dimensiones):
        self.nombre = nombre
        self.q = load_file(archivo_q, dimensiones)
        self.r = load_file(archivo_r, dimensiones)
        print("Dataset {}: Q={} R={}".format(
            self.nombre, self.q.shape, self.r.shape))

class Curva:   
    def __init__(self):
        self.precisiones = []
        self.eficiencias = []

### Creacion de los objetos de los dataset MEL y VGG

In [2]:
dataset_MEL = Dataset("MEL", "dataset_MEL/MEL-Q-128_float32.bin", "dataset_MEL/MEL-R-128_float32.bin", 128)
dataset_VGG = Dataset("VGG", "dataset_VGG/VGG-Q-4096_float32.bin", "dataset_VGG/VGG-R-4096_float32.bin", 4096)

Dataset MEL: Q=(32000, 128) R=(10000, 128)
Dataset VGG: Q=(1000, 4096) R=(10000, 4096)


### Los siguientes comandos "importan" las clases que van a servir para realizar el analisís de la tarea

In [3]:
%run "PCA.ipynb"
%run "Trees.ipynb"

### Evaluación del dataset MEL

In [4]:
ev1_mel = EvaluarArbol(dataset_MEL)
ev1_mel.linear_scan()
curva_kd1  = ev1_mel.evaluar_arbol(algorithm="kdtree", trees=1)
curva_kd10 = ev1_mel.evaluar_arbol(algorithm="kdtree", trees=10)
curva_kd30 = ev1_mel.evaluar_arbol(algorithm="kdtree", trees=30)
curva_km3  = ev1_mel.evaluar_arbol(algorithm="kmeans", branching=3)
curva_km30 = ev1_mel.evaluar_arbol(algorithm="kmeans", branching=30)
curva_km90 = ev1_mel.evaluar_arbol(algorithm="kmeans", branching=90)
ev2_mel = EvaluarPCA(dataset_MEL)
ev2_mel.calcular_PCA()
for dims in 2, 4, 8, 16, 24, 32, 48, 64, 80, 96, 112, 128:
    ev2_mel.reducir_y_buscar(dims)

MEL linear scan de Q=(32000, 128)xR=(10000, 128)
MEL linear scan = 26.0 segundos
MEL construcción kdtree-1 = 0.09 segundos
  kdtree-1	precision=3.8%	tiempo=0.4% checks=1
  kdtree-1	precision=17.7%	tiempo=0.5% checks=10
  kdtree-1	precision=35.9%	tiempo=1.4% checks=40
  kdtree-1	precision=44.9%	tiempo=2.5% checks=70
  kdtree-1	precision=51.1%	tiempo=3.3% checks=100
  kdtree-1	precision=63.4%	tiempo=5.8% checks=200
  kdtree-1	precision=70.6%	tiempo=7.7% checks=300
  kdtree-1	precision=75.4%	tiempo=11.0% checks=400
  kdtree-1	precision=78.8%	tiempo=13.4% checks=500
  kdtree-1	precision=81.6%	tiempo=16.3% checks=600
  kdtree-1	precision=83.7%	tiempo=19.5% checks=700
  kdtree-1	precision=85.4%	tiempo=25.6% checks=800
  kdtree-1	precision=87.0%	tiempo=36.4% checks=900
  kdtree-1	precision=88.3%	tiempo=40.0% checks=1000
  kdtree-1	precision=99.4%	tiempo=182.7% checks=5000
MEL construcción kdtree-10 = 0.21 segundos
  kdtree-10	precision=4.1%	tiempo=0.6% checks=1
  kdtree-10	precision=28.2%	tie

NameError: name 'ev2' is not defined

### Evaluación del dataset VGG

In [ ]:
ev1_vgg = EvaluarArbol(dataset_VGG)
ev1_vgg.linear_scan()
curva_kd1  = ev1_vgg.evaluar_arbol(algorithm="kdtree", trees=1)
curva_kd10 = ev1_vgg.evaluar_arbol(algorithm="kdtree", trees=10)
curva_kd30 = ev1_vgg.evaluar_arbol(algorithm="kdtree", trees=30)
curva_km3  = ev1_vgg.evaluar_arbol(algorithm="kmeans", branching=3)
curva_km30 = ev1_vgg.evaluar_arbol(algorithm="kmeans", branching=30)
curva_km90 = ev1_vgg.evaluar_arbol(algorithm="kmeans", branching=90)
ev2_vgg = EvaluarPCA(dataset_VGG)
ev2_vgg.calcular_PCA()
for dims in 2, 4, 8, 16, 24, 32, 48, 64, 80, 96, 112, 128:
    ev2_vgg.reducir_y_buscar(dims)

### Generación de las gráficas para el dataset MEL

### Generación de las gráficas para el dataset VGG

## Analisís 

- Caso A: El desempeño la búsqueda aproximada logra al menos un 95% de efectividad con el menor tiempo posible.

- Caso B: El desempeño la búsqueda aproximada logra al menos un 50% de efectividad con el menor tiempo posible.

## Preguntas

1. ¿Qué configuración de Randomized KD-Tree logra el mejor resultado para el caso A? (mencionar número de árboles y hojas visitadas) ¿y para el caso B?
- MEL
- VGG

2. ¿Qué configuración de Hierarchical K-Means Tree logra el mejor resultado para el caso A? (mencionar branching y hojas visitadas) ¿y para el caso B?
- MEL
- VGG

3. ¿Qué configuración de Linear Scan sobre PCA logra el mejor resultado para el caso A? (mencionar número de dimensiones y varianza retenida) ¿y para el caso B?
- MEL
- VGG

4. En resumen, ¿qué técnica logró el mejor resultado para el caso A? ¿y para el caso B?
- MEL
- VGG



